[https://docs.arbor-sim.org/en/stable/tutorial/network_ring.html](https://docs.arbor-sim.org/en/stable/tutorial/network_ring.html)

In [10]:
import arbor
import numpy as np

In [11]:
def make_cable_cell(gid):
    # (1) Build a segment tree
    tree = arbor.segment_tree()

    # Soma (tag=1) with radius 6 μm, modelled as cylinder of length 2*radius
    s = tree.append(arbor.mnpos, arbor.mpoint(-12, 0, 0, 6), arbor.mpoint(0, 0, 0, 6), tag=1)

    # Single dendrite (tag=3) of length 50 μm and radius 2 μm attached to soma.
    b0 = tree.append(s, arbor.mpoint(0, 0, 0, 2), arbor.mpoint(50, 0, 0, 2), tag=3)

    # Attach two dendrites (tag=3) of length 50 μm to the end of the first dendrite.
    # Radius tapers from 2 to 0.5 μm over the length of the dendrite.
    b1 = tree.append(b0, arbor.mpoint(50, 0, 0, 2), arbor.mpoint(50+50/np.sqrt(2), 50/np.sqrt(2), 0, 0.5), tag=3)
    # Constant radius of 1 μm over the length of the dendrite.
    b2 = tree.append(b0, arbor.mpoint(50, 0, 0, 1), arbor.mpoint(50+50/np.sqrt(2), -50/np.sqrt(2), 0, 1), tag=3)

    # Associate labels to tags
    labels = arbor.label_dict()
    labels['soma'] = '(tag 1)'
    labels['dend'] = '(tag 3)'
    
    # (2) Mark location for synapse at the midpoint of branch 1 (the first dendrite).
    labels['synapse_site'] = '(location 1 0.5)'
    # Mark the root of the tree.
    labels['root'] = '(root)'
    
    # (3) Create a decor and a cable_cell
    decor = arbor.decor()

    # Put hh dynamics on soma, and passive properties on the dendrites.
    decor.paint('"soma"', arbor.density('hh'))
    decor.paint('"dend"', arbor.density('pas'))

    # (4) Attach a single synapse.
    decor.place('"synapse_site"', arbor.synapse('expsyn'), 'syn')

    # Attach a spike detector with threshold of -10 mV.
    decor.place('"root"', arbor.spike_detector(-10), 'detector')

    cell = arbor.cable_cell(tree, labels, decor)

    return cell

In [12]:
# (5) Create a recipe that generates a network of connected cells.
class ring_recipe (arbor.recipe):

    def __init__(self, ncells):
        # The base C++ class constructor must be called first, to ensure that
        # all memory in the C++ class is initialized correctly.
        arbor.recipe.__init__(self)
        self.ncells = ncells
        self.props = arbor.neuron_cable_properties()

    # (6) The num_cells method that returns the total number of cells in the model
    # must be implemented.
    def num_cells(self):
        return self.ncells

    # (7) The cell_description method returns a cell
    def cell_description(self, gid):
        return make_cable_cell(gid)

    # The kind method returns the type of cell with gid.
    # Note: this must agree with the type returned by cell_description.
    def cell_kind(self, gid):
        return arbor.cell_kind.cable

    # (8) Make a ring network. For each gid, provide a list of incoming connections.
    def connections_on(self, gid):
        src = (gid-1)%self.ncells
        w = 0.01 # 0.01 μS on expsyn
        d = 5 # ms delay
        return [arbor.connection((src,'detector'), 'syn', w, d)]

    # (9) Attach a generator to the first cell in the ring.
    def event_generators(self, gid):
        if gid==0:
            sched = arbor.explicit_schedule([1]) # one event at 1 ms
            weight = 0.1 # 0.1 μS on expsyn
            return [arbor.event_generator('syn', weight, sched)]
        return []

    # (10) Place a probe at the root of each cell.
    def probes(self, gid):
        return [arbor.cable_probe_membrane_voltage('"root"')]

    def global_properties(self, kind):
        return self.props

In [13]:
# (11) Instantiate recipe
ncells = 4
recipe = ring_recipe(ncells)

# (12) Create a default execution context, domain decomposition and simulation
context = arbor.context()
decomp = arbor.partition_load_balance(recipe, context)
sim = arbor.simulation(recipe, decomp, context)

# (13) Set spike generators to record
sim.record(arbor.spike_recording.all)

# (14) Attach a sampler to the voltage probe on cell 0. Sample rate of 10 sample every ms.
handles = [sim.sample((gid, 0), arbor.regular_schedule(0.1)) for gid in range(ncells)]

# (15) Run simulation for 100 ms
sim.run(100)
print('Simulation finished')

# (16) Print spike times
for sp in sim.spikes():
    print(' ', sp)

# (17) Plot the recorded voltages over time.


Simulation finished
  ((0, 0), 1.44555691)
  ((1, 0), 8.77866787)
  ((2, 0), 15.9908923)
  ((3, 0), 23.16931239)
  ((0, 0), 30.42039981)
  ((1, 0), 37.63181929)
  ((2, 0), 44.84353983)
  ((3, 0), 52.05502328)
  ((0, 0), 59.26679204)
  ((1, 0), 66.47864598)
  ((2, 0), 73.69041421)
  ((3, 0), 80.90223546)
  ((0, 0), 88.1139914)
  ((1, 0), 95.32579538)
